In [1]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from sklearn.preprocessing import StandardScaler
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression

In [4]:
onehot1 = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

onehot2 = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder=StandardScaler())

hybrid_vars = [0,1, 2, 3, 4, 5, 6, 7, -3, -2, -1]
hybrid_model = TimeSeriesGradientBoosting(model1 = LinearRegression(), model2 = XGBRegressor(), model1_variables = hybrid_vars)

hybrid_xgb_pipe = Pipeline([
    ('preprocessor', onehot1),
    ('regressor', hybrid_model)
])

sgd_pipe = Pipeline([
    ('preprocessor', onehot2),
    ('regressor', SGDRegressor())
])

In [5]:
def full_run():
    products = []
    customers = []
    for i in range(2):
        product = TimeSeriesGenerator(size=365, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=365, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithms=[hybrid_xgb_pipe, sgd_pipe], algorithm_name=["Gradient boosted decision tree", "Linear regression"], online=[False, False])
    return thesis


In [6]:
thesis = full_run()

Finished experiment! Elapsed time: 0.4468052387237549, total Elapsed time: 0.4471287727355957, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Important
Finished experiment! Elapsed time: 0.4998810291290283, total Elapsed time: 0.947009801864624, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Half observed, importance: Important
Finished experiment! Elapsed time: 0.4949605464935303, total Elapsed time: 1.4419703483581543, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Unobserved, importance: Important
Finished experiment! Elapsed time: 0.48282647132873535, total Elapsed time: 1.9247968196868896, Algorithm: Gradient boosted decision tree Type: Sudden Drift, Dropped variables: None, magnitude: Small, Drift time: Fully observed, importance: Medium
Finished

In [7]:
results = thesis.resultsToDF()

In [8]:
results[["Algorithm","SMAPE", "Drift type", "Drift time", "Drift magnitude"]].groupby(["Algorithm","Drift type", "Drift time", "Drift magnitude"]).aggregate(["mean", "std", "min", "max"])

SMAPE  \
                                                                                      mean   
Algorithm                      Drift type        Drift time     Drift magnitude              
Gradient boosted decision tree Incremental Drift Fully observed Large             6.371667   
                                                                Small             3.210000   
                                                 Half observed  Large            13.585833   
                                                                Small             4.264167   
                                                 Unobserved     Large            11.234167   
                                                                Small             3.917500   
                               No Drift          No Drift       No Drift          2.432500   
                               Sudden Drift      Fully observed Large             6.214167   
                                                                Small             3.218333   
                                                 Half observed  Large             8.189167   
                                                                Small             3.418333   
                                                 Unobserved     Large            15.545000   
                                                                Small             4.717500   
                               Sudden Shock      3 days         Large             3.157500   
                                                                Small             2.570000   
                                                 7 days         Large             3.185000   
                                                                Small             2.530000   
Linear regression              Incremental Drift Fully observed Large             4.104167   
                                                                Small             3.165833   
                                                 Half observed  Large            18.700833   
                                                                Small             5.836667   
                                                 Unobserved     Large            14.964167   
                                                                Small             5.359167   
                               No Drift          No Drift       No Drift          3.095000   
                               Sudden Drift      Fully observed Large             4.245000   
                                                                Small             3.199167   
                                                 Half observed  Large             5.369167   
                                                                Small             3.333333   
                                                 Unobserved     Large            20.214167   
                                                                Small             6.222500   
                               Sudden Shock      3 days         Large             3.227500   
                                                                Small             3.102500   
                                                 7 days         Large             3.257500   
                                                                Small             3.105000   

                                                                                            \
                                                                                       std   
Algorithm                      Drift type        Drift time     Drift magnitude              
Gradient boosted decision tree Incremental Drift Fully observed Large             3.289573   
                                                                Small             0.789108   
                                                 Half observed  Large             9.053830   
                                          

In [9]:
results.sort_values("SMAPE", ascending=False)   

,Algorithm,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,SMAPE
301,Linear regression,6.0,Sudden Drift,Large,Medium,Unobserved,4143.123925,31.70
216,Linear regression,0.0,Sudden Drift,Large,Important,Unobserved,2360.479840,27.87
93,Gradient boosted decision tree,3.0,Sudden Drift,Large,Important,Unobserved,2281.430268,27.67
52,Gradient boosted decision tree,0.0,Sudden Drift,Large,Important,Unobserved,2238.780370,27.42
219,Linear regression,0.0,Sudden Drift,Large,Medium,Unobserved,2880.438664,27.09
...,...,...,...,...,...,...,...,...
106,Gradient boosted decision tree,3.0,Incremental Drift,Small,Unimportant,Fully observed,16.604115,2.30
76,Gradient boosted decision tree,0.0,Incremental Drift,Large,Unimportant,Unobserved,16.463564,2.28
7,Gradient boosted decision tree,NaN,Sudden Drift,Small,Unimportant,Half observed,15.249314,2.28
58,Gradient boosted decision tree,0.0,Sudden Drift,Large,Unimportant,Unobserved,15.719719,2.25
